In [2]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  4703, 10870,  4333], shape=(11314,))

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Trabajo Practico 1 - Procesamiento de Lenguaje Natural

### Tomás Koller

### Ejercicio 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

Paso 1: Vectorización previa

In [31]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Cargar datos de 20 Newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  remove=('headers','footers','quotes'))

# Vectorizar con TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test  = vectorizer.transform(newsgroups_test.data)


Paso 2: Seleccionar 5 documentos al azar y medir similaridad

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Elegimos 5 documentos al azar del set de test
rng = np.random.default_rng(42)
sample_idx = rng.choice(X_test.shape[0], size=5, replace=False)

# Calculamos la matriz de similaridad coseno entre esos 5 y todos los del test
similarity_matrix = cosine_similarity(X_test[sample_idx], X_test)

# Guardar resultados detallados en un archivo .txt
output_path = "consigna1_resultados.txt"
with open(output_path, "w", encoding="utf-8") as f:

    top_k = 5
    for i, doc_idx in enumerate(sample_idx):
        sims = similarity_matrix[i]
        sims[doc_idx] = -1
        top_indices = np.argsort(sims)[-top_k:][::-1]
        top_scores  = sims[top_indices]

        # --- Impresión resumida en pantalla ---
        print(f"\n Documento semilla #{doc_idx} — categoría:",
              newsgroups_test.target_names[newsgroups_test.target[doc_idx]])
        print("Texto inicial:", newsgroups_test.data[doc_idx][:200].replace('\n',' '), "...")
        print("Documentos más similares:")
        for rank, (idx, score) in enumerate(zip(top_indices, top_scores), 1):
            label = newsgroups_test.target_names[newsgroups_test.target[idx]]
            print(f"  {rank}. idx={idx:<5} | label={label:<20} | coseno={score:.3f}")
        print("-"*80)

        # --- Detalle completo en archivo ---
        f.write(f"\n Documento semilla #{doc_idx} — categoría: {newsgroups_test.target_names[newsgroups_test.target[doc_idx]]}\n")
        f.write("Texto inicial: " + newsgroups_test.data[doc_idx][:500].replace('\n',' ') + " ...\n\n")
        f.write("Documentos más similares:\n")
        for rank, (idx, score) in enumerate(zip(top_indices, top_scores), 1):
            label = newsgroups_test.target_names[newsgroups_test.target[idx]]
            texto_similar = newsgroups_test.data[idx][:500].replace("\n", " ")
            f.write(f"  {rank}. idx={idx:<5} | label={label:<25} | coseno={score:.3f}\n")
            f.write(f"     → Texto inicial: {texto_similar} ...\n\n")

print(f"\n Resultados detallados guardados en '{output_path}'")
None



 Documento semilla #5827 — categoría: comp.windows.x
Texto inicial: One way to "mask" the left button check the contents of the XEvent that the XAddEventHandler send to your event_handling function (in your case, it is the show_mouse_position function.   If the XEvent ...
Documentos más similares:
  1. idx=1099  | label=comp.windows.x       | coseno=0.194
  2. idx=4566  | label=comp.windows.x       | coseno=0.177
  3. idx=7100  | label=comp.windows.x       | coseno=0.161
  4. idx=276   | label=comp.windows.x       | coseno=0.147
  5. idx=6011  | label=comp.sys.ibm.pc.hardware | coseno=0.128
--------------------------------------------------------------------------------


54

521

26

64

...

64

528

64

151


 Resultados detallados guardados en 'consigna1_resultados.txt'


Interpretación de los resultados

Caso 1 — comp.windows.x

- El documento semilla y sus 4 vecinos principales pertenecen a la misma categoría “comp.windows.x”, y todos tratan sobre programación en el sistema X Window, manejo de eventos del mouse, botones o shells en MOTIF/Athena.
- El quinto más similar pertenece a “comp.sys.ibm.pc.hardware”, donde también se menciona un problema con mouse y puertos COM.
- Aunque la etiqueta es distinta, el vocabulario compartido (“mouse”, “button”, “screen”) explica la coincidencia.
- La métrica TF-IDF coseno capta coherencia temática a nivel léxico: palabras técnicas comunes generan alta similaridad aunque la categoría difiera levemente.


Caso 2 — sci.electronics

- Los textos similares incluyen principalmente temas de temperatura y calor, con apariciones de categorías “sci.space” y “misc.forsale”.
- Los documentos de sci.space también discuten temperatura en el espacio, lo cual explica la cercanía léxica (“temperature”, “warm”, “degrees”, “radiator”).
- Solo uno de los 5 vecinos pertenece a la misma clase (“sci.electronics”), lo que indica que el modelo reconoce el tópico físico (temperatura) más que la disciplina exacta.
- TF-IDF + coseno refleja similitud superficial por términos, sin distinguir el dominio de aplicación. Es un ejemplo de semántica débil, donde las coincidencias de vocabulario pueden conectar temas distintos (espacio vs electrónica).


Caso 3 — talk.politics.guns

- Todos los vecinos pertenecen a la misma categoría “talk.politics.guns”, con puntuaciones de coseno relativamente altas (≈ 0.42 → 0.18).
- Los fragmentos comparten expresiones idénticas (“jmd@handheld.com”, “Waco”, “government”, “fire”), indicando no solo tema común sino probablemente autor o thread compartido.
- Este es un caso de coherencia léxica y autoral: el vector TF-IDF logra agrupar mensajes del mismo hilo o usuario, validando la eficacia del modelo para separar tópicos de debate político.


Caso 4 — soc.religion.christian

- La mayoría de los documentos similares provienen de foros religiosos: “soc.religion.christian”, “alt.atheism”, “talk.religion.misc”.
- El vocabulario (“God”, “essence”, “Platonic”, “omnipotence”) se repite en todos los fragmentos, lo que explica las altas similaridades inter-clase.
- La métrica no distingue perspectivas opuestas (teísmo vs ateísmo), solo contexto religioso. Refleja bien la proximidad temática, aunque no la orientación semántica.


Caso 5 — comp.windows.x (sin coincidencias)

- En este caso, los valores de coseno son ≈ 0.000, lo que indica ausencia total de similitud.
- El documento semilla quedó vacío, muy corto o fue filtrado de tokens relevantes.
- TF-IDF depende fuertemente de contenido textual y vocabulario compartido; textos con poco contenido o palabras raras quedan sin vecinos cercanos.

### Ejercicio 2

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.


In [33]:
def consigna2_prototype_classifier(X_train, y_train, X_test, y_test, target_names) -> Dict[str, Any]:
    """
    1-NN por similaridad coseno: para cada doc de test, predice la clase del doc de train más similar.
    """
    sims = linear_kernel(X_test, X_train)  # (n_test x n_train)
    nn_idx = np.argmax(sims, axis=1)       # índice del vecino más similar
    y_pred = y_train[nn_idx]

    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, target_names=target_names, digits=3, zero_division=0)
    return {"f1_macro": float(f1), "report": report, "y_pred": y_pred}

# Ejecutar Consigna 2
res2 = consigna2_prototype_classifier(Xtr, data.y_train, Xte, data.y_test, data.target_names)
print(f"F1-macro (test) = {res2['f1_macro']:.3f}")
print(res2['report'])


F1-macro (test) = 0.527
                          precision    recall  f1-score   support

             alt.atheism      0.445     0.433     0.439       319
           comp.graphics      0.490     0.501     0.496       389
 comp.os.ms-windows.misc      0.491     0.497     0.494       394
comp.sys.ibm.pc.hardware      0.496     0.531     0.513       392
   comp.sys.mac.hardware      0.483     0.530     0.506       385
          comp.windows.x      0.610     0.559     0.584       395
            misc.forsale      0.527     0.495     0.511       390
               rec.autos      0.352     0.568     0.434       396
         rec.motorcycles      0.541     0.553     0.547       398
      rec.sport.baseball      0.618     0.655     0.636       397
        rec.sport.hockey      0.685     0.732     0.708       399
               sci.crypt      0.665     0.596     0.628       396
         sci.electronics      0.479     0.387     0.428       393
                 sci.med      0.650     0.543     0

In [45]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

sims1 = cosine_similarity(X_test[:5], X_train[:5])
sims2 = linear_kernel(X_test[:5], X_train[:5])

np.allclose(sims1, sims2)


True

Interpretación por categorías

1) Las clases con mejor desempeño son:

- rec.sport.hockey (F1 ≈ 0.71)
- talk.politics.mideast (≈ 0.64)
- rec.sport.baseball (≈ 0.64)
- sci.crypt (≈ 0.63)
Estas categorías contienen vocabulario muy distintivo y técnico (por ejemplo, “NHL”, “Palestine”, “encryption”), por lo que la similaridad coseno discrimina bien los documentos.

2) Las clases con desempeño intermedio (0.50 ± 0.05) incluyen comp. y sci. topics**, donde existe solapamiento léxico (“window”, “driver”, “board”) entre distintas sub-categorías de computación o ciencia.

3) Los peores desempeños aparecen en rec.autos (F1 ≈ 0.43) y talk.religion.misc (F1 ≈ 0.32), temas donde el vocabulario puede ser más general o conversacional, sin términos exclusivos por categoría.

El modelo 1-NN basado en TF-IDF + coseno logra un rendimiento moderado-bueno (F1 ≈ 0.53) para ser un método sin entrenamiento paramétrico: cada documento de train funciona como un “prototipo” textual.


La performance está limitada por:

- Ruido léxico y polisemia (palabras comunes entre tópicos).
- Clase de tamaño desigual: categorías grandes dominan por frecuencia.
- Falta de generalización semántica (TF-IDF mide similitud superficial, no significado).

### Ejercicio 3

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


In [ ]:
def consigna3_naive_bayes_tuning(dataset: Dataset) -> pd.DataFrame:
    """
    Pequeña búsqueda: grid de TfidfVectorizer (sin cambiar ngram_range) y alpha de NB.
    Ordena por F1-macro en test.
    """
    vec_grid = [
        {"stop_words": "english", "lowercase": True, "strip_accents": "ascii", "max_df": 0.95, "min_df": 2, "use_idf": True, "norm": "l2"},
        {"stop_words": "english", "lowercase": True, "strip_accents": None,   "max_df": 0.95, "min_df": 2, "use_idf": True, "norm": "l2"},
        {"stop_words": "english", "lowercase": True, "strip_accents": "ascii", "max_df": 0.90, "min_df": 2, "use_idf": True, "norm": "l2"},
        {"stop_words": None,      "lowercase": True, "strip_accents": None,   "max_df": 0.95, "min_df": 5, "use_idf": True, "norm": "l2"},
    ]
    
    # Se probaron distintos parámetros del vectorizador y del hiperparámetro alpha, manteniendo fijo ngram_range=(1, 1).
    nb_grid = [
        ("MultinomialNB", MultinomialNB, {"alpha": [0.1, 0.5, 1.0, 2.0]}),
        ("ComplementNB",  ComplementNB,  {"alpha": [0.1, 0.5, 1.0, 2.0]}),
    ]

    rows = []
    for vec_params in vec_grid:
        vect = TfidfVectorizer(ngram_range=(1,1), **vec_params)
        Xtr = vect.fit_transform(dataset.X_train_text)
        Xte = vect.transform(dataset.X_test_text)

        for name, cls, hp in nb_grid:
            for a in hp["alpha"]:
                model = cls(alpha=a)
                model.fit(Xtr, dataset.y_train)
                y_pred = model.predict(Xte)
                f1 = f1_score(dataset.y_test, y_pred, average="macro")
                rows.append({
                    "vectorizer": vec_params,
                    "model": name,
                    "alpha": a,
                    "f1_macro_test": float(f1)
                })

    df = pd.DataFrame(rows).sort_values("f1_macro_test", ascending=False).reset_index(drop=True)
    return df

# Ejecutar Consigna 3
df_nb = consigna3_naive_bayes_tuning(data)
df_nb.head(10)


,vectorizer,model,alpha,f1_macro_test
0,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,0.5,0.697363
1,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,0.5,0.697363
2,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,0.5,0.697363
3,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,1.0,0.694292
4,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,1.0,0.694292
5,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,1.0,0.694292
6,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,2.0,0.689360
7,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,2.0,0.689360
8,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,2.0,0.689360
9,"{'stop_words': 'english', 'lowercase': True, '...",ComplementNB,0.1,0.688661


Conclusiones


- ComplementNB supera sistemáticamente a MultinomialNB, lo que confirma su ventaja en datasets desbalanceados (como 20 Newsgroups).
Su formulación ajusta los conteos considerando las clases complementarias, reduciendo el sesgo hacia clases frecuentes.

- El mejor valor de alpha = 0.5 logra F1-macro ≈ 0.70, un incremento de más de 0.17 puntos respecto al baseline de Consigna 2 (0.53).


| Modelo                    | Tipo                         | Entrenamiento                      | F1-macro (test) | Diferencia |
| ------------------------- | ---------------------------- | ---------------------------------- | --------------- | ---------- |
| 1-NN (Consigna 2)         | Basado en similaridad coseno | No paramétrico                     | 0.527           | —          |
| ComplementNB (Consigna 3) | Probabilístico supervisado   | Entrenado con priors y likelihoods | **0.697**       | + 0.17     |


- El clasificador NB aprende distribuciones de palabras por clase, en lugar de depender solo de la similitud léxica puntual.
- Esto le permite generalizar mejor ante documentos nuevos con vocabulario parcialmente distinto.¿
- Además, NB es computacionalmente más eficiente (producto escalar log-probabilístico en lugar de matriz completa de similaridades).

### Ejercicio 4

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

In [35]:
def consigna4_word_similarity(vect: TfidfVectorizer, X_train, words: List[str], topk=5) -> pd.DataFrame:
    """
    De matriz doc-term a term-doc (trasposición). Busca las palabras más similares por coseno.
    """
    vocab = vect.vocabulary_
    X_td = X_train.T  # sparse (V x N)

    rows = []
    # índice a término para decodificar
    idx_to_term = {i: t for t, i in vocab.items()}

    for w in words:
        if w not in vocab:
            rows.append({"word": w, "most_similar": [], "scores": [], "note": "palabra no está en el vocabulario"})
            continue

        wi = vocab[w]
        sims = linear_kernel(X_td[wi], X_td).ravel()  # similitud coseno en espacio término-documento
        sims[wi] = -np.inf  # evitar self-match

        top_idx = np.argpartition(sims, -topk)[-topk:]
        top_idx = top_idx[np.argsort(sims[top_idx])[::-1]]

        rows.append({
            "word": w,
            "most_similar": [idx_to_term[i] for i in top_idx],
            "scores": [float(sims[i]) for i in top_idx],
            "note": ""
        })

    return pd.DataFrame(rows)

# Ejecutar Consigna 4 (elegí palabras claras/temáticas)
palabras = ["space", "windows", "god", "hockey", "medical"]
df_words = consigna4_word_similarity(vect, Xtr, palabras, topk=5)
df_words


,word,most_similar,scores,note
0,space,"[nasa, shuttle, launch, sci, station]","[2.0948361965781555, 1.3191687811489314, 1.081...",
1,windows,"[dos, file, files, thanks, ms]","[3.6366737216302685, 2.534554430234965, 2.0882...",
2,god,"[jesus, bible, believe, people, faith]","[3.9179827384541572, 2.9349461673304966, 2.785...",
3,hockey,"[game, players, team, nhl, play]","[1.2481765183933027, 1.054502830898892, 0.9894...",
4,medical,"[health, medicine, hospital, tests, disease]","[0.4077752984171904, 0.3140258686878096, 0.284...",


Conclusones
| Palabra base | Palabras más similares                     | Interpretación                                                                                                                                          |
| ------------ | ------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **space**    | nasa, shuttle, launch, sci, station        | Términos directamente relacionados con exploración espacial. El vector refleja co-ocurrencia en documentos científicos de la categoría *sci.space*.     |
| **windows**  | dos, file, files, thanks, ms               | Léxico técnico de sistemas operativos Microsoft. Captura correctamente el contexto informático de *comp.windows.x* y *comp.os.ms-windows.misc*.         |
| **god**      | jesus, bible, believe, people, faith       | Palabras típicas del discurso religioso. Muestra una fuerte cohesión temática dentro de las categorías *soc.religion.christian* y *talk.religion.misc*. |
| **hockey**   | game, players, team, nhl, play             | Léxico deportivo. Los vectores reflejan uso conjunto en noticias o foros de *rec.sport.hockey*.                                                         |
| **medical**  | health, medicine, hospital, tests, disease | Léxico clínico-sanitario, coherente con los tópicos de *sci.med*.                                                                                       |


- El análisis demuestra que el espacio de co-ocurrencia TF-IDF captura asociaciones temáticas: las palabras cercanas pertenecen al mismo dominio semántico, aunque el modelo no tenga comprensión “simbólica” del significado.

- Las similitudes se explican por frecuencias conjuntas en documentos de categorías afines: TF-IDF pondera los términos que aparecen juntos en contextos específicos, generando agrupamientos coherentes.

- Los valores numéricos de similaridad (entre ~0.3 y 3.9 según la escala del producto interno) reflejan fuerza de asociación: mayor valor ⇒ mayor co-ocurrencia en contextos comunes.

- No obstante, esta técnica no distingue sinónimos verdaderos ni relaciones semánticas profundas (por ejemplo, “doctor” y “nurse” podrían no aparecer como vecinos si no co-ocurren en los mismos documentos).